In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import numpy as np

STOPWORDS = set(stopwords.words('english'))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
file_name_raw_data_pkl = 'apr_may_ind_mar_may_ire1.pkl'
df = pd.read_pickle(file_name_raw_data_pkl)

In [3]:
# df_Ind = df[df.country == 'India']

In [4]:
df.shape

(12088, 9)

In [5]:
df.head()

,author,title,description,url,urlToImage,publishedAt,content,source.name,country
0,RTÉ News,HSE to double Covid-19 test capacity,"The Health Service Executive has said the capacity to test for the cornavirus will be doubled from this week, to around 4,500 tests a day.",https://www.rte.ie/news/coronavirus/2020/0406/1128703-coronavirus-ireland/,https://img.rasset.ie/0013fea7-1600.jpg,2020-04-06T06:22:55Z,"The Health Service Executive has said the capacity to test for the cornavirus will be doubled from this week, to around 4,500 tests a day. \r\nA network of laboratories and 50 community test centres have been set up across the country. \r\nThe HSE said it has man… [+1043 chars]",RTE,Ireland
1,Ailbhe Conneely,PCI Moderator calls for hope for those with Covid-19,The Moderator of the Presbyterian Church in Ireland has called for hope for those suffering from Covid-19 this Easter.,https://www.rte.ie/news/2020/0406/1128835-pci-moderator-calls-for-hope-for-those-with-covid-19/,https://img.rasset.ie/001410ff-1600.jpg,2020-04-06T12:56:31Z,"The Moderator of the Presbyterian Church in Ireland has called for hope for those suffering from Covid-19 this Easter. \r\nRt Rev Dr William Henry says the peak number of confirmed virus cases is estimated to occur over the Easter weekend, which is the peak of … [+1011 chars]",RTE,Ireland
2,RTÉ News,Twelfth of July parades cancelled due to Covid-19,The Orange Order has announced that this year's annual Twelfth of July parades have been cancelled due to Covid-19.,https://www.rte.ie/news/ulster/2020/0406/1128808-coronavirus-orange-parades/,https://img.rasset.ie/00125bb0-1600.jpg,2020-04-06T11:37:52Z,The Orange Order has announced that this year's annual Twelfth of July parades have been cancelled due to Covid-19.\r\nGrand Master Edward Stevenson said the decision was taken in light of the ongoing pandemic and follows consultation with the Grand Masters of … [+1517 chars],RTE,Ireland
3,RTÉ News,Tiger at Bronx Zoo tests positive for Covid-19,A tiger at New York's Bronx Zoo has tested positive for COVID-19 and it is believed to have contracted the virus from a caretaker who was asymptomatic at the time.,https://www.rte.ie/news/coronavirus/2020/0406/1128701-tiger-bronx-zoo/,https://img.rasset.ie/00141024-1600.jpg,2020-04-06T06:09:32Z,"A tiger at New York's Bronx Zoo has tested positive for COVID-19, the institution said over the weekend.\r\nIt is believed to have contracted the virus from a caretaker who was asymptomatic at the time.\r\nThe four-year-old Malayan tiger named Nadia along with he… [+2122 chars]",RTE,Ireland
4,RTÉ News,COVID-19: US facing toughest week- surgeon general,"The United States is entering what a senior official warned would be the ""hardest"" week of the coronavirus crisis.",https://www.rte.ie/news/2020/0406/1128696-virus-us-surgeon-deaths/,https://img.rasset.ie/0010e38f-1600.jpg,2020-04-06T03:47:16Z,"The United States is entering what a senior official warned would be the ""hardest"" week of the coronavirus crisis.\r\nWith health authorities recording 1,200 new deaths from the virus in a 24 hour period, the Surgeon General, Jerome Adams, said the country is … [+1999 chars]",RTE,Ireland


In [6]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12088 entries, 0 to 10
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   author       11262 non-null  object
 1   title        12088 non-null  object
 2   description  12087 non-null  object
 3   url          12088 non-null  object
 4   urlToImage   11568 non-null  object
 5   publishedAt  12088 non-null  object
 6   content      11190 non-null  object
 7   source.name  12088 non-null  object
 8   country      12088 non-null  object
dtypes: object(9)
memory usage: 944.4+ KB


## Format Date Time 

In [7]:
df['publishedAt'][:5]

0    2020-04-06T06:22:55Z
1    2020-04-06T12:56:31Z
2    2020-04-06T11:37:52Z
3    2020-04-06T06:09:32Z
4    2020-04-06T03:47:16Z
Name: publishedAt, dtype: object

In [8]:
df['published_at'] =  pd.to_datetime(df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ').dt.date
df['published_at'][:5]

0    2020-04-06
1    2020-04-06
2    2020-04-06
3    2020-04-06
4    2020-04-06
Name: published_at, dtype: object

### There are - on an average - 1.5 to 2 sentences apart from the first line available.

## NLP FUNCTIONS

### Tokenizer

In [9]:
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string

STOPWORDS = set(stopwords.words('english'))

def text_tokenizer(text):
    try:
        if text:
            sequences = sent_tokenize(text.split('…')[0])
            seq_tokens = [word_tokenize(seq) for seq in sequences]
            no_punct_seq_tokens = []
            for seq_token in seq_tokens:
                no_punct_seq_tokens.append([token for token in seq_token if token not in string.punctuation and token.lower() not in STOPWORDS])
            return no_punct_seq_tokens
        else:
            return None
    except:
        print(text)
        return None

### Improved Token Cleaner

Improvements taken from Wordcloud Generation observations and errors.

In [10]:
def clean_tokens(token_sequences):
    cleaned_token_sequences = []
    if token_sequences:
        for token_list in token_sequences:
            tokens = [x.strip().lower() for x in token_list if x != 'char' and x.strip().lower() not in STOPWORDS 
                          and x.strip() not in string.punctuation and x.strip() not in ['’', '‘' , '–', '\'s', 'n\'t' , '’', '``', "''"] 
                          and '…' not in x and '+' not in x and len(x) != 1]
            tokens = ['covid-19' if x == 'covid' else x for x in tokens ]
            tokens = ['coronavirus' if x == 'corona' else x for x in tokens]
            tokens = ['government' if x == 'govt' else x for x in tokens]
            tokens = ['modi' if x == 'namo' else x for x in tokens]
            cleaned_token_sequences.append(tokens)
        return cleaned_token_sequences
    else:
        return None

### Intializing POS tagger

In [11]:
import os
from nltk.tag import StanfordPOSTagger

path = os.getcwd()
path_to_stnfrd_core_nlp = '/Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/'

jar = path_to_stnfrd_core_nlp + 'stanford-postagger.jar'
model = path_to_stnfrd_core_nlp + 'models/english-bidirectional-distsim.tagger'

st = StanfordPOSTagger(model, jar, encoding='utf8')

### Lemmatization

In [12]:
from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

lm = WordNetLemmatizer()

dict_pos_map = {
    # Look for NN in the POS tag because all nouns begin with NN
    'NN': NOUN,
    # Look for VB in the POS tag because all nouns begin with VB
    'VB':VERB,
    # Look for JJ in the POS tag because all nouns begin with JJ
    'JJ' : ADJ,
    # Look for RB in the POS tag because all nouns begin with RB
    'RB':ADV  
}

def get_lemma(no_punct_seq_tokens):
    try:
        if no_punct_seq_tokens:
            normalized_sequence = []
            for each_seq in st.tag_sents(sentences=no_punct_seq_tokens):
                normalized_tokens = []
                for tuples in each_seq:
                    temp = tuples[0]
                    if tuples[1] == "NNP" or tuples[1] == "NNPS":
                        continue
                    if tuples[1][:2] in dict_pos_map.keys():
                        temp = lm.lemmatize(tuples[0].lower(), 
                                            pos=dict_pos_map[tuples[1][:2]])
                    normalized_tokens.append(temp)
                normalized_sequence.append(normalized_tokens)
            return normalized_sequence
        else:
            return None
    except:
        print(no_punct_seq_tokens)
        return None

In [13]:
def pre_process(df, col_list):
    for col in col_list:
        print('Begin: ' + col)
        col_tokenized = col + '_tokenized'
        col_normalised = col + '_normalized'
        col_tokenized_cleaned = col + '_tokens_cleaned'
        # tokenizer removes punctuations and stopwords
        df[col_tokenized] = df[col].apply(lambda x : text_tokenizer(x))
        df[col_tokenized_cleaned] = df[col_tokenized].apply(lambda x : clean_tokens(x))
        df[col_normalised] = df[col_tokenized_cleaned].apply(lambda x: get_lemma(x))
        print('End: ' + col)

# Execution

In [14]:
# for demo
# df_demo = df_Ind[:5]
from datetime import datetime

print("Start: " + str(datetime.now()))

pre_process(df, ['content', 'title', 'description'])

print("End: " + str(datetime.now()))

Start: 2020-05-18 09:36:27.829459
Being: content
Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.8 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:87)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.ta

Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.6 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:87)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagCoreLabelsOrHasWords(

Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.8 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:87)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagCoreLabelsOrHasWords(

Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.5 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:87)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagCoreLabelsOrHasWords(

Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.8 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:87)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagCoreLabelsOrHasWords(

Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.8 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:130)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagCoreLabelsOrHasWords

Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.5 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:129)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagCoreLabelsOrHasWords

Loading default properties from tagger /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger
Loading POS tagger from /Users/f75k8bx/Documents/MyGitHub/Medium_Blog_Codes/stanford-postagger/models/english-bidirectional-distsim.tagger ... done [0.5 sec].
Exception in thread "main" java.lang.OutOfMemoryError: Java heap space
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:129)
	at edu.stanford.nlp.sequences.ExactBestSequenceFinder.bestSequence(ExactBestSequenceFinder.java:37)
	at edu.stanford.nlp.tagger.maxent.TestSentence.runTagInference(TestSentence.java:341)
	at edu.stanford.nlp.tagger.maxent.TestSentence.testTagInference(TestSentence.java:328)
	at edu.stanford.nlp.tagger.maxent.TestSentence.tagSentence(TestSentence.java:151)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagSentence(MaxentTagger.java:1052)
	at edu.stanford.nlp.tagger.maxent.MaxentTagger.tagCoreLabelsOrHasWords

In [15]:
# df_demo.head()

### Checkpoint - Save the DF to save time

In [17]:
df.to_pickle(file_name_raw_data_pkl + 'processed.pkl')

In [ ]:
#df = pd.read_pickle('demo.pkl')

# Thanks for visiting!